In [ ]:
!pip install -q PyMuPDF beautifulsoup4 requests transformers spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 30.8 MB/s eta 0:00:00


In [ ]:
# import spacy
# from transformers import pipeline
# import fitz  # PyMuPDF
# import re
# import requests
# from bs4 import BeautifulSoup
# import json

# # Load spaCy model
# nlp = spacy.load("en_core_web_sm")

# # Load the question-generation pipeline using a T5 model
# question_generator = pipeline("text2text-generation", model="valhalla/t5-base-qg-hl")

# def generate_qa_pairs(text):
#     doc = nlp(text)
#     sentences = [sent.text for sent in doc.sents]

#     qa_pairs = []
#     for sentence in sentences:
#         # Ensure the sentence length does not exceed the model's maximum input length
#         inputs = f"generate question: {sentence}"
#         if len(inputs.split()) > 512:
#             inputs = " ".join(inputs.split()[:512])

#         question = question_generator(inputs)
#         qa_pairs.append((question[0]['generated_text'], sentence))

#     return qa_pairs

# def extract_text_from_pdf(pdf_path):
#     document = fitz.open(pdf_path)
#     text = ""
#     for page_num in range(len(document)):
#         page = document.load_page(page_num)
#         text += page.get_text()
#     # Remove headers, footers, and page numbers
#     text = re.sub(r'\s*Computer Security Fundamentals\s*', '', text)
#     text = re.sub(r'\s*Chuck Easttom\s*', '', text)
#     text = re.sub(r'\s*\d+\s*', '', text)
#     text = re.sub(r'\.+\s*', '', text)  # remove dots
#     text = re.sub(r'Copyright.*?All rights reserved\.', '', text, flags=re.DOTALL)
#     text = re.sub(r'Trademarks.*?\.', '', text, flags=re.DOTALL)
#     text = re.sub(r'ISBN-.*?\n', '', text)
#     text = re.sub(r'Printed in.*?\n', '', text)
#     text = re.sub(r'First Printing:.*?\n', '', text)
#     text = re.sub(r'\s+', ' ', text)
#     text = text.replace('\n', ' ').replace('\r', '')  # replace new lines and carriage returns

#     return text

# def scrape_website(url):
#     response = requests.get(url)
#     soup = BeautifulSoup(response.content, 'html.parser')
#     paragraphs = soup.find_all('p')
#     text = " ".join([para.get_text() for para in paragraphs])
#     return text

# # # Lists of URLs and PDF paths
# # pdf_paths = [
# #     '/content/drive/MyDrive/Chatbot/fdocuments.net_computer-security-fundamentals book.pdf',
# #     '/path/to/another/book.pdf'
# # ]
# website_urls = [
#     'https://www.proofpoint.com/us/threat-reference/phishing',

#     'https://www.kaspersky.com/resource-center/definitions/what-is-cyber-security',
#     'https://www.cisco.com/site/us/en/learn/topics/security/what-is-cybersecurity.html',
#     'https://www.verizon.com/about/account-security/phishing',
#     'https://www.cisco.com/c/en/us/products/security/email-security/what-is-phishing.html',
#     'https://www.cisco.com/c/en/us/products/security/email-security/what-is-phishing.html#~how-phishing-works',
#     'https://www.cisco.com/c/en/us/products/security/email-security/what-is-phishing.html#~ai-and-phishing',
#     'https://www.cisco.com/c/en/us/products/security/email-security/what-is-phishing.html#~phishing-awareness',
#     'https://www.cisco.com/c/en/us/products/security/email-security/what-is-phishing.html#~phishing-attacks',
#     'https://www.cloudflare.com/learning/access-management/phishing-attack/'
#     'https://www.checkpoint.com/cyber-hub/cyber-security/what-is-cybersecurity/',
#     'https://cybermagazine.com/cyber-security/history-cybersecurity',
#     'https://www.knowledgehut.com/blog/security/history-of-cyber-security',
#     'https://www.monroecollege.edu/news/cybersecurity-history-hacking-data-breaches',
#     'https://www.linkedin.com/pulse/brief-history-cybersecurity-robert-cristello/'
# ]

# # Collect QA pairs from all sources
# all_qa_pairs = {
#    # "pdf": [],
#     "website": []
# }

# # # Process PDF files
# # for pdf_path in pdf_paths:
# #     pdf_text = extract_text_from_pdf(pdf_path)
# #     qa_pairs_from_pdf = generate_qa_pairs(pdf_text)
# #     all_qa_pairs["pdf"].extend(qa_pairs_from_pdf)

# # Process websites
# for url in website_urls:
#     website_text = scrape_website(url)
#     qa_pairs_from_website = generate_qa_pairs(website_text)
#     all_qa_pairs["website"].extend(qa_pairs_from_website)

# # Save the QA pairs to a JSON file
# with open('qa_dataset.json', 'w') as f:
#     json.dump(all_qa_pairs, f)

# #print(f"QA pairs from PDF: {all_qa_pairs['pdf']}")
# print(f"QA pairs from websites: {all_qa_pairs['website']}")


In [ ]:
import spacy
from transformers import pipeline
import fitz  # PyMuPDF
import re
import requests
from bs4 import BeautifulSoup
import json
import csv

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Load the question-generation pipeline using a T5 model
question_generator = pipeline("text2text-generation", model="valhalla/t5-base-qg-hl")

def clean_text(text):
    # Replace problematic characters with their correct forms
    text = text.replace("â€™", "'")
    text = text.replace("â€“", "–")
    text = text.replace("Â", "")
    text = text.replace("�", "")
    # Remove leading and trailing spaces and newlines
    text = text.strip()
    return text

def generate_qa_pairs(text):
    doc = nlp(text)
    sentences = [clean_text(sent.text) for sent in doc.sents]

    qa_pairs = []
    for sentence in sentences:
        # Ensure the sentence length does not exceed the model's maximum input length
        inputs = f"generate question: {sentence}"
        if len(inputs.split()) > 512:
            inputs = " ".join(inputs.split()[:512])

        question = question_generator(inputs)
        # Clean both question and answer
        question_text = clean_text(question[0]['generated_text'])
        answer_text = clean_text(sentence)
        qa_pairs.append((question_text, answer_text))

    return qa_pairs

def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    # Remove headers, footers, and page numbers
    text = re.sub(r'\s*Computer Security Fundamentals\s*', '', text)
    text = re.sub(r'\s*Chuck Easttom\s*', '', text)
    text = re.sub(r'\s*\d+\s*', '', text)
    text = re.sub(r'\.+\s*', '', text)  # remove dots
    text = re.sub(r'Copyright.*?All rights reserved\.', '', text, flags=re.DOTALL)
    text = re.sub(r'Trademarks.*?\.', '', text, flags=re.DOTALL)
    text = re.sub(r'ISBN-.*?\n', '', text)
    text = re.sub(r'Printed in.*?\n', '', text)
    text = re.sub(r'First Printing:.*?\n', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('\n', ' ').replace('\r', '')  # replace new lines and carriage returns

    return clean_text(text)

def scrape_website(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    text = " ".join([para.get_text() for para in paragraphs])
    return clean_text(text)

# # Lists of URLs and PDF paths
# pdf_paths = [
#     '/content/drive/MyDrive/Chatbot/fdocuments.net_computer-security-fundamentals book.pdf',
#     '/path/to/another/book.pdf'
# ]
website_urls = [
    'https://www.proofpoint.com/us/threat-reference/phishing',
    'https://www.kaspersky.com/resource-center/definitions/what-is-cyber-security',
    'https://www.cisco.com/site/us/en/learn/topics/security/what-is-cybersecurity.html',
    'https://www.verizon.com/about/account-security/phishing'
]

# Collect QA pairs from all sources
all_qa_pairs = {
   # "pdf": [],
    "website": []
}

# # Process PDF files
# for pdf_path in pdf_paths:
#     pdf_text = extract_text_from_pdf(pdf_path)
#     qa_pairs_from_pdf = generate_qa_pairs(pdf_text)
#     all_qa_pairs["pdf"].extend(qa_pairs_from_pdf)

# Process websites
for url in website_urls:
    website_text = scrape_website(url)
    qa_pairs_from_website = generate_qa_pairs(website_text)
    all_qa_pairs["website"].extend(qa_pairs_from_website)

# Save the QA pairs to a CSV file
with open('qa_dataset1.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Question", "Answer"])
    for qa_pair in all_qa_pairs["website"]:
        writer.writerow(qa_pair)

# Save the QA pairs to a JSON file (optional)
with open('qa_dataset1.json', 'w') as f:
    json.dump(all_qa_pairs, f)

print(f"QA pairs from websites saved to 'qa_dataset.csv'")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


QA pairs from websites saved to 'qa_dataset.csv'
